In [ ]:
import numpy as np
#Correct
#basic model of no accidental death and no mutations
class Bacterium:
    count = 0  # Iterator for unique IDs
    i_b = 1 #rate for reproduction
    i_d = 1/40 #rate for death
    a_d = 0.0000009 #accidental death rate
    
    def __init__(self, x_b, x_d):
        self.id = Bacterium.count  # Unique identity for each bacterium
        Bacterium.count += 1
        self.x_b = x_b  # Give birth until this time
        self.x_d = x_d  # Cannot die before this time
        self.age = 0
        self.death_event_id = 0

    #define representation
    def __repr__(self):
        return f'bacteria number: {self.id}, age: {self.age}'
    
    def reproduce_time(self):
        """Return the time for the next reproduction event."""
        return np.random.exponential(1 / Bacterium.i_b)
    
    def death_time(self):
        """Return the time for the next death event."""
        return np.random.exponential(1 / Bacterium.i_d)
    
    def reproduce(self):
        """
        Generate a child bacterium with parameters drawn from a normal distribution
        around the parent's parameters.
        """
        return Bacterium(self.x_b, self.x_d) 
    
    def reproduce_mut(self, std_dev=1, prob = 1/2): #prob gives the probability that the child mutates
        """
        Generate a child bacterium with parameters drawn from a normal distribution
        around the parent's parameters.
        """
        rand = np.random.rand()
        if rand < prob: 
            new_x_b = np.random.normal(self.x_b, std_dev)
            new_x_d = np.random.normal(self.x_d, std_dev)
            if new_x_b >= 0 and new_x_d>= 0:
                return Bacterium(new_x_b, new_x_d) #returns the bacteria with the mutations
            else: 
                return None #the bacteria will survive only if both x_b and x_d are non-negative
        else: #if rand >= prob, then no mutations for children
            return Bacterium(self.x_b, self.x_d)
        
    
    def accidental_death_time(self, population): #calculate the exponential of accidental death
        """
        Returns accidental death rate, this depends on the current population.
        """
        return np.random.exponential(1 / (population*Bacterium.a_d))

#inefficient version of basic reprduction, don't use
def simulate_bacteria_population(initial_population, max_time):
    """Simulate bacteria population dynamics."""
    population = [Bacterium(30, 20) for _ in range(initial_population)]  # Initial population
    time = 0
    while time < max_time and population:
        # Get times for the next birth and death events for each bacterium
        events = []
        for bacterium in population: #CHANGE: can't die from the start, stops giving birth after x_b
            birth_time = bacterium.reproduce_time() 
            death_time = bacterium.death_time() 
            if bacterium.age + birth_time <= bacterium.x_b:
                events.append((birth_time, 'birth', bacterium))
                bacterium.age += birth_time
            elif bacterium.age >= bacterium.x_d:
                events.append((death_time, 'death', bacterium))
                bacterium.age += death_time
            
        
        # Find the earliest event
        next_event = min(events, key=lambda x: x[0]) #KEY TAKES IN A FUNCTION AND USES IT TO EXTRACT THE ELEMENT TO BE COMPARED
        event_time, event_type, bacterium = next_event
        
        # Update simulation time
        time += event_time
        if time >= max_time:
            break  # Stop if max_time is reached
        
        if event_type == 'birth':
            # Reproduce a new bacterium with parameters sampled from a normal distribution
            new_bacterium = bacterium.reproduce(std_dev=0.01) #WE ARE NOT USING STANDARD DEVIATION YET
            population.append(new_bacterium)
        elif event_type == 'death':
            # Remove the bacterium from the population
            population.remove(bacterium)
        
        print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {bacterium.id}, Population: {len(population)}")
    
    return population



In [ ]:
#Basic reproduction model, more efficient model, uncomment print statements if you want but it'll be less efficient. 
def basic_simulation(initial_population, max_time, xb_0, xd_0):
    Bacterium.count = 0
    times = [0]   # Track times of events
    time = 0
    population = []
    for _ in range(initial_population):
        population.append(Bacterium(xb_0, xd_0))

    while time < max_time and population:
        pop = len(population)
        individual_rate = Bacterium.i_b + Bacterium.i_d #no accidental deaths
        rate = pop * individual_rate
        event_time = np.random.exponential(1/rate) #beta = 1/lambda recuerdalo joder
        rand = np.random.rand()*rate
        index = int(np.floor(rand/individual_rate))
        bacteria = population[index]
        event_type = None
        if rand <= index * individual_rate + Bacterium.i_b:
            if bacteria.age + event_time > bacteria.x_b:
                continue
            for bacterium in population: #add time to bacteria age
                bacterium.age += event_time
            event_type = 'birth'
            
            new_bacterium = bacteria.reproduce()
            population.append(new_bacterium)
            times.append(time)
            #print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {Bacterium.count - 1}, Population: {len(population)}")
                
        elif rand <= index * individual_rate + Bacterium.i_b + Bacterium.i_d:
            if bacteria.age + event_time < bacteria.x_d:
                continue
            for bacterium in population: #add time to bacteria age
                bacterium.age += event_time
            event_type = 'death'
            population.remove(bacteria)
            #print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {bacteria.id}, Population: {len(population)}, Age: {bacteria.age}") 
        time += event_time
return population, times

In [ ]:
#Efficient implementation of population with mutations and accidental deaths, uncomment print statements if you want but it'll make it less efficient. 
def simulation_probability(initial_population, max_time, xb_0, xd_0):
    Bacterium.count = 0
    xbs = [xb_0]  # Track x_b values
    xds = [xd_0]  # Track x_d values
    times = [0]   # Track times of events
    time = 0
    population = []
    for _ in range(initial_population):
        population.append(Bacterium(xb_0, xd_0))

    while time < max_time and population:
        pop = len(population)
        individual_rate = Bacterium.i_b + Bacterium.i_d + pop * Bacterium.a_d
        rate = pop * individual_rate
        event_time = np.random.exponential(1/rate) #beta = 1/lambda recuerdalo joder
        rand = np.random.rand()*rate
        index = int(np.floor(rand/individual_rate))
        bacteria = population[index]
        event_type = None
        if rand <= index * individual_rate + Bacterium.i_b:
            if bacteria.age + event_time > bacteria.x_b:
                continue
            for bacterium in population: #add time to bacteria age
                bacterium.age += event_time
            event_type = 'birth'
            
            new_bacterium = bacteria.reproduce_mut(std_dev=0.05, prob=0.1)
            if new_bacterium:
                population.append(new_bacterium)
                times.append(time)
                xbs.append(new_bacterium.x_b)
                xds.append(new_bacterium.x_d)
                #print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {Bacterium.count - 1}, Population: {len(population)}")
                
        elif rand <= index * individual_rate + Bacterium.i_b + Bacterium.i_d:
            if bacteria.age + event_time < bacteria.x_d:
                continue
            for bacterium in population: #add time to bacteria age
                bacterium.age += event_time
            event_type = 'death'
            population.remove(bacteria)
            #print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {bacteria.id}, Population: {len(population)}, Age: {bacteria.age}")
            
        elif rand <= (index + 1) * individual_rate:
            event_type = 'accident'
            for bacterium in population: #add time to bacteria age
                bacterium.age += event_time
            population.remove(bacteria)
            #print(f"Time: {time:.2f}, Event: {event_type}, Bacterium ID: {bacteria.id}, Population: {len(population)}, Age: {bacteria.age}")
        
        time += event_time
        
    return population, xbs, xds, times